# VERIFICATION TESTING

# HER2 One Scanner - Aperio FDA

- 5-Fold (80/20) split, No Holdout Set
- Truth = Categorical from Mean of 7 continuous scores 
- Epoch at automatic Stop when loss<.001 change 
- LeNet model, 10 layers, Dropout (0.7)

In [1]:
import numpy as np
import pandas as pd
import random
from keras.callbacks import EarlyStopping
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report
import csv
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#For single scanner
BASE_PATH = '/home/diam/Desktop/1Scanner_VerificationTest_HER2data/Aperio_FDA/'
#BASE PATH for working from home:
#BASE_PATH = '/home/OSEL/Desktop/HER2_data_categorical/'
#epochs = 10
batch_size = 32
num_classes = 3
#epochs = 35

## Get Data - Practice

In [3]:
#This is the version from Ravi's code:

#FDA
#X_FDA = []
#idx_FDA = []
#for index, image_filename in list(enumerate(BASE_PATH)):
#	img_file = cv2.imread(BASE_PATH + '/' + image_filename)
#	if img_file is not None:
		#img_file = smisc.imresize(arr = img_file, size = (600,760,3))
#		img_file = smisc.imresize(arr = img_file, size = (120,160,3))		
#		img_arr = np.asarray(img_file)
#		X_FDA.append(img_arr)
#		idx_FDA.append(index)

#X_FDA = np.asarray(X_FDA)
#idx_FDA = np.asarray(idx_FDA)
#random.seed(rs)
#random_id = random.sample(idx_FDA, len(idx_FDA)/2)
#random_FDA = []
#for i in random_id:
#	random_FDA.append(X_FDA[i])

#random_FDA = np.asarray(random_FDA)

## Get Data - Real

In [4]:
def get_data(folder):
    X = []
    y = []
    filenames = []

    for hclass in os.listdir(folder):
        if not hclass.startswith('.'):
            if hclass in ["1"]:
                label = 1
            else: #label must be 1 or 2
                if hclass in ["2"]:
                    label = 2
                else:
                    label = 3
            for image_filename in os.listdir(folder + hclass):
                filename = folder + hclass + '/' + image_filename
                img_file = cv2.imread(folder + hclass + '/' + image_filename)
                
                if img_file is not None:
                    img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                    filenames.append(filename)

    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(filenames)
    return X,y,filenames

In [5]:
X, y, z = get_data(BASE_PATH)


#print(X)
#print(y)
#print(z)
print(len(X))
print(len(y))
print(len(z))

#INTEGER ENCODE
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
encoder = LabelEncoder()
y_cat = np_utils.to_categorical(encoder.fit_transform(y))
print(y_cat)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


241
241
241
[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.

### Old Code

In [6]:

#encoder = LabelEncoder()
#encoder.fit(y)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#encoded_y_train = encoder.transform(y_train)
#encoded_y_test = encoder.transform(y_test)

#y_train = np_utils.to_categorical(encoded_y_train)
#y_test = np_utils.to_categorical(encoded_y_test)

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Fit Model with K-Fold X-Val

In [8]:
kf = KFold(n_splits = 5, random_state=5, shuffle=True)
print(kf.get_n_splits(y))
print(kf)


#for train_index, test_index in kf.split(y):
#    X_train, X_test = X[train_index], X[test_index]
#    print(train_index, test_index)




5
KFold(n_splits=5, random_state=5, shuffle=True)


In [9]:

oos_y = []
oos_pred = []
fold = 0


for train, test in kf.split(y_cat):
    fold+=1
    print("fold #{}".format(fold))
        
    X_train = X[train]
    y_train = y_cat[train]
    X_test = X[test]
    y_test = y_cat[test]
    
    #encoder = LabelEncoder()
    #encoder.fit(y_test)
    #y_train = np_utils.to_categorical(encoder.transform(y_train))
    #y_test = np_utils.to_categorical(encoder.transform(y_test))
    
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')
    
    model.fit(
            X,
            y_cat,
            validation_data=(X_test,y_test),
            callbacks=[monitor],
            shuffle=True,
            batch_size=batch_size,
            verbose=1,
            epochs=1000)
        
    pred = model.predict(X_test)
        
    oos_y.append(y_test)
    pred = np.argmax(pred,axis=1)
    oos_pred.append(pred)
    
        #measure the fold's accuracy
    y_compare = np.argmax(y_test,axis=1) #for accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print("Fold Score (accuracy): {}".format(score))
    print(pred)

fold #1
Train on 241 samples, validate on 49 samples
Epoch 1/1000
241/241 [==============================] - 2s 9ms/step - loss: 1.1057 - acc: 0.4440 - val_loss: 0.8117 - val_acc: 0.5714
Epoch 2/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.9021 - acc: 0.5394 - val_loss: 1.6858 - val_acc: 0.3673
Epoch 3/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8580 - acc: 0.6763 - val_loss: 0.5942 - val_acc: 0.8571
Epoch 4/1000
241/241 [==============================] - 0s 1ms/step - loss: 1.0406 - acc: 0.6971 - val_loss: 0.6987 - val_acc: 0.7347
Epoch 5/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6451 - acc: 0.7967 - val_loss: 0.5804 - val_acc: 0.7959
Epoch 6/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8148 - acc: 0.7137 - val_loss: 0.5538 - val_acc: 0.8367
Epoch 7/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.5798 - acc: 0.7884 - val_loss: 0.6335 - val_acc: 0.7143
Epoch 8/1

241/241 [==============================] - 0s 1ms/step - loss: 0.7258 - acc: 0.8174 - val_loss: 0.2503 - val_acc: 0.8980
Epoch 62/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2295 - acc: 0.9129 - val_loss: 0.2254 - val_acc: 0.8980
Epoch 63/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2608 - acc: 0.9087 - val_loss: 0.2173 - val_acc: 0.8980
Epoch 64/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2207 - acc: 0.9046 - val_loss: 0.1690 - val_acc: 0.8980
Epoch 65/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1929 - acc: 0.9129 - val_loss: 0.4493 - val_acc: 0.8980
Epoch 66/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2485 - acc: 0.8963 - val_loss: 0.1813 - val_acc: 0.8980
Epoch 67/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.6059 - acc: 0.8091 - val_loss: 0.2624 - val_acc: 0.8980
Epoch 68/1000
241/241 [==============================] - 0s 1ms/step 

241/241 [==============================] - 0s 2ms/step - loss: 0.1716 - acc: 0.9461 - val_loss: 0.0407 - val_acc: 0.9796
Epoch 122/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0619 - acc: 0.9793 - val_loss: 0.0218 - val_acc: 1.0000
Epoch 123/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0312 - acc: 0.9917 - val_loss: 0.0144 - val_acc: 1.0000
Epoch 124/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0302 - acc: 0.9959 - val_loss: 0.0149 - val_acc: 1.0000
Epoch 125/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0509 - acc: 0.9834 - val_loss: 0.2894 - val_acc: 0.8980
Epoch 126/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0908 - acc: 0.9793 - val_loss: 0.0711 - val_acc: 0.9796
Epoch 127/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6192 - acc: 0.9046 - val_loss: 0.0360 - val_acc: 1.0000
Epoch 128/1000
241/241 [==============================] - 0s 2m

241/241 [==============================] - 0s 1ms/step - loss: 0.0234 - acc: 1.0000 - val_loss: 0.0050 - val_acc: 1.0000
Epoch 182/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0394 - acc: 0.9876 - val_loss: 0.0031 - val_acc: 1.0000
Epoch 183/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0412 - acc: 0.9793 - val_loss: 0.0037 - val_acc: 1.0000
Epoch 184/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5259 - acc: 0.9170 - val_loss: 0.0223 - val_acc: 1.0000
Epoch 185/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0649 - acc: 0.9834 - val_loss: 0.0040 - val_acc: 1.0000
Epoch 186/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0229 - acc: 0.9917 - val_loss: 0.0030 - val_acc: 1.0000
Epoch 187/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0244 - acc: 0.9917 - val_loss: 0.0020 - val_acc: 1.0000
Epoch 188/1000
241/241 [==============================] - 0s 1m

241/241 [==============================] - 0s 1ms/step - loss: 0.3382 - acc: 0.9004 - val_loss: 0.2957 - val_acc: 0.8958
Epoch 45/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4184 - acc: 0.8548 - val_loss: 0.2273 - val_acc: 0.9167
Epoch 46/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3761 - acc: 0.8589 - val_loss: 0.2277 - val_acc: 0.9167
Epoch 47/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3828 - acc: 0.8755 - val_loss: 0.2114 - val_acc: 0.9167
Epoch 48/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3349 - acc: 0.8921 - val_loss: 0.4580 - val_acc: 0.8125
Epoch 49/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4959 - acc: 0.8382 - val_loss: 0.2196 - val_acc: 0.9375
Epoch 50/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3287 - acc: 0.8963 - val_loss: 0.2129 - val_acc: 0.9167
Epoch 51/1000
241/241 [==============================] - 0s 1ms/step 

...

241/241 [==============================] - 0s 2ms/step - loss: 0.7502 - acc: 0.8838 - val_loss: 0.0504 - val_acc: 0.9792
Epoch 84/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1002 - acc: 0.9544 - val_loss: 0.0374 - val_acc: 1.0000
Epoch 85/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5581 - acc: 0.8714 - val_loss: 0.1037 - val_acc: 0.9792
Epoch 86/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1224 - acc: 0.9461 - val_loss: 0.0433 - val_acc: 0.9792
Epoch 87/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0894 - acc: 0.9710 - val_loss: 0.0477 - val_acc: 0.9792
Epoch 88/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3767 - acc: 0.9004 - val_loss: 0.0497 - val_acc: 0.9792
Epoch 89/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0982 - acc: 0.9585 - val_loss: 0.0276 - val_acc: 1.0000
Epoch 90/1000
241/241 [==============================] - 0s 1ms/step 

241/241 [==============================] - 1s 3ms/step - loss: 1.1796 - acc: 0.4481 - val_loss: 0.9241 - val_acc: 0.5208
Epoch 2/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8927 - acc: 0.5477 - val_loss: 1.0522 - val_acc: 0.5208
Epoch 3/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9498 - acc: 0.5353 - val_loss: 0.9137 - val_acc: 0.7708
Epoch 4/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8872 - acc: 0.5768 - val_loss: 0.8774 - val_acc: 0.5625
Epoch 5/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8849 - acc: 0.6349 - val_loss: 0.7997 - val_acc: 0.8125
Epoch 6/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7768 - acc: 0.6639 - val_loss: 0.6793 - val_acc: 0.7292
Epoch 7/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7438 - acc: 0.7344 - val_loss: 0.6619 - val_acc: 0.7292
Epoch 8/1000
241/241 [==============================] - 0s 1ms/step - loss:

Epoch 62/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2479 - acc: 0.8963 - val_loss: 0.2780 - val_acc: 0.8542
Epoch 63/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2359 - acc: 0.8963 - val_loss: 0.2385 - val_acc: 0.8542
Epoch 64/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2952 - acc: 0.8921 - val_loss: 0.2271 - val_acc: 0.8542
Epoch 65/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.5495 - acc: 0.8299 - val_loss: 0.4442 - val_acc: 0.8542
Epoch 66/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2886 - acc: 0.9046 - val_loss: 0.3004 - val_acc: 0.8542
Epoch 67/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2079 - acc: 0.9087 - val_loss: 0.2565 - val_acc: 0.8542
Epoch 68/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2921 - acc: 0.8797 - val_loss: 0.3259 - val_acc: 0.7917
Epoch 69/1000
241/241 [==============================] 

241/241 [==============================] - 0s 1ms/step - loss: 0.1137 - acc: 0.9751 - val_loss: 0.0606 - val_acc: 0.9792
Epoch 123/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0546 - acc: 0.9751 - val_loss: 0.0286 - val_acc: 1.0000
Epoch 124/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0385 - acc: 0.9917 - val_loss: 0.0253 - val_acc: 1.0000
Epoch 125/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0479 - acc: 0.9834 - val_loss: 0.0337 - val_acc: 1.0000
Epoch 126/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0344 - acc: 0.9834 - val_loss: 0.0165 - val_acc: 1.0000
Epoch 127/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5761 - acc: 0.9170 - val_loss: 0.0485 - val_acc: 1.0000
Epoch 128/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0396 - acc: 0.9876 - val_loss: 0.0315 - val_acc: 1.0000
Epoch 129/1000
241/241 [==============================] - 0s 1m

241/241 [==============================] - 0s 1ms/step - loss: 0.0112 - acc: 1.0000 - val_loss: 0.0014 - val_acc: 1.0000
Epoch 183/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0195 - acc: 0.9917 - val_loss: 0.0203 - val_acc: 1.0000
Epoch 184/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0298 - acc: 0.9834 - val_loss: 9.0004e-04 - val_acc: 1.0000
Epoch 185/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0920 - acc: 0.9876 - val_loss: 0.0058 - val_acc: 1.0000
Epoch 186/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0074 - acc: 1.0000 - val_loss: 0.0016 - val_acc: 1.0000
Epoch 187/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7266 - acc: 0.9295 - val_loss: 0.0396 - val_acc: 1.0000
Epoch 188/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0405 - acc: 0.9751 - val_loss: 0.0027 - val_acc: 1.0000
Epoch 189/1000
241/241 [==============================] - 0